# 第二课：图游走类算法习题

本节实践主要涉及到DeepWalk和Node2Vec的关键代码，目的是让同学们能够进一步理解、使用以及根据自身需求修改这些模块。

In [1]:
# 安装依赖
# !pip install paddlepaddle==1.8.5
!pip install pgl

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 7.9MB 14.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 20.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 27.1MB/s eta 0:00:01


## 1. DeepWalk采样算法

Graph类的实现可参考 PGL/pgl/graph.py，DeepWalk的代码详见 ./deepwalk.py

	NOTE：对于给定的节点，DeepWalk会等概率的选取下一个相邻节点加入路径，直至达到最大路径长度，或者没有下一个节点可选。

<img src="https://ai-studio-static-online.cdn.bcebos.com/159e470f09bb4e12bae080a4733d46d0861a08e812e643d5b8b7f080b16f2e38" width="85%" height="85%" />

请实现Graph类的random_walk函数

In [2]:
%%writefile userdef_graph.py
from pgl.graph import Graph

import numpy as np

class UserDefGraph(Graph):
    def random_walk(self, nodes, walk_len):
        """
        输入：nodes - 当前节点id list (batch_size,)
             walk_len - 最大路径长度 int
        输出：以当前节点为起点得到的路径 list (batch_size, walk_len)

        用到的函数
        1. self.successor(nodes)
           描述：获取当前节点的下一个相邻节点id列表
           输入：nodes - list (batch_size,)
           输出：succ_nodes - list of list ((num_successors_i,) for i in range(batch_size))
        2. self.outdegree(nodes)
           描述：获取当前节点的出度
           输入：nodes - list (batch_size,)
           输出：out_degrees - list (batch_size,)
        """
        walks = [[node] for node in nodes]

        walks_ids = np.arange(0, len(nodes))
        cur_nodes = np.array(nodes)
        for l in range(walk_len):
            """选取有下一个节点的路径继续采样，否则结束"""
            outdegree = self.outdegree(cur_nodes)
            walk_mask = (outdegree != 0)
            if not np.any(walk_mask):
               break
            cur_nodes = cur_nodes[walk_mask]
            walks_ids = walks_ids[walk_mask]
            outdegree = outdegree[walk_mask]

            ######################################
            # 请在此补充代码采样出下一个节点
            succ_nodes = self.successor(cur_nodes)

            sample_index = np.floor(np.random.rand(outdegree.shape[0]) * outdegree).astype("int64")
            next_nodes = []
            for s, ind, walk_id in zip(succ_nodes, sample_index, walks_ids):
                walks[walk_id].append(s[ind])
                next_nodes.append(s[ind])

            ######################################
            cur_nodes = np.array(next_nodes)
        return walks

Writing userdef_graph.py


In [3]:
!python my_deepwalk.py --use_my_random_walk --epoch 5 # 用自己实现的random walk训练DeepWalk模型，可在 ./tmp/deepwalk/walks/ 中查看构造的节点路径
!python link_predict.py --ckpt_path ./tmp/deepwalk/paddle_model --epoch 100 #测试

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
[INFO] 2020-11-25 13:01:52,146 [my_deepwalk.py:  274]:	Namespace(batch_size=512, epoch=5, hidden_size=128, neg_num=20, processes=2, save_path='./tmp/deepwalk', use_my_random_walk=True, walk_len=5, win_size=5)
[INFO] 2020-11-25 13:01:53,120 [my_deepwalk.py:  192]:	Start random walk on disk...
[INFO] 2020-11-25 13:01:53,950 [my_deepwalk.py:  203]:	Random walk on disk Done.
2020-11-25 13:01:53,952-WARNING: paddle.fluid.layers.py_reader() may be deprecated in the near future. Please use paddle.fluid.io.DataLoader.from_generator() instead.
[INFO] 2020-11-25 13:01:54,763 [my_deepwalk.py:  250]:	Step 1 DeepWalk Loss: 0.723897  0.568443 s/step.
[INFO] 2020-11-25 13:01:57,646 [my_deepwalk.py:  250]:	Step 10 DeepWalk Loss: 0.7129

## 2. SkipGram模型训练

	NOTE：在得到节点路径后，node2vec会使用SkipGram模型学习节点表示，给定中心节点，预测局部路径中还有哪些节点。模型中用了negative sampling来降低计算量。

<img src="https://ai-studio-static-online.cdn.bcebos.com/5ee18998f2c84598a01a43aad15270f154f837dc972747e3aa69d6c2eb7d5d10" width="85%" height="85%" />

请你实现一下loss的计算过程吧。可参考 PGL/examples/node2vec/node2vec.py 中的 node2vec_model 函数

In [4]:
%%writefile userdef_model.py
import paddle.fluid.layers as l

def userdef_loss(embed_src, weight_pos, weight_negs):
    """
    输入：embed_src   - 中心节点向量 list (batch_size, 1, embed_size)
         weight_pos  - 标签节点向量 list (batch_size, 1, embed_size)
         weight_negs - 负样本节点向量 list (batch_size, neg_num, embed_size)
    输出：loss - 正负样本的交叉熵 float
    """
    
    ##################################
    # 请在这里实现SkipGram的loss计算过程
    pos_logits = l.matmul(
        embed_src, weight_pos, transpose_y=True)  # [batch_size, 1, 1]
    neg_logits = l.matmul(
        embed_src, weight_negs, transpose_y=True)  # [batch_size, 1, neg_num]

    ones_label = pos_logits * 0. + 1.
    ones_label.stop_gradient = True
    pos_loss = l.sigmoid_cross_entropy_with_logits(pos_logits, ones_label)

    zeros_label = neg_logits * 0.
    zeros_label.stop_gradient = True
    neg_loss = l.sigmoid_cross_entropy_with_logits(neg_logits, zeros_label)
    
    loss = (l.reduce_mean(pos_loss) + l.reduce_mean(neg_loss)) / 2
    ##################################
    return loss

Writing userdef_model.py


接下来看看在ArXiv数据集上的效果吧~

In [5]:
!python my_node2vec.py  --use_my_model --epoch 5 # 使用自己定义的loss函数
!python link_predict.py --ckpt_path ./tmp/node2vec/paddle_model --epoch 100

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
[INFO] 2020-11-25 13:13:04,391 [my_node2vec.py:  393]:	Namespace(batch_size=512, dataset='ArXiv', epoch=5, hidden_size=128, neg_num=20, offline_learning=False, p=0.25, processes=2, q=0.25, save_path='./tmp/node2vec', use_cuda=False, use_my_model=True, use_my_sample=False, walk_len=5, win_size=5)
2020-11-25 13:13:05,435-WARNING: paddle.fluid.layers.py_reader() may be deprecated in the near future. Please use paddle.fluid.io.DataLoader.from_generator() instead.
[INFO] 2020-11-25 13:13:06,248 [my_node2vec.py:  358]:	Step 1 Node2vec Loss: 0.724371  0.541657 s/step.
[INFO] 2020-11-25 13:13:09,104 [my_node2vec.py:  358]:	Step 10 Node2vec Loss: 0.717879  0.269546 s/step.
[INFO] 2020-11-25 13:13:11,851 [my_node2vec.py:  358]:	S

## 3. Node2Vec采样算法


	NOTE：Node2Vec会根据与上个节点的距离按不同概率采样得到当前节点的下一个节点。

<img src="https://ai-studio-static-online.cdn.bcebos.com/09001163a1064101a8dd2892eb559cf2006aa93d7fe84c70b2ad47b810f4c86a" width="85%" height="85%" />

PGL/pgl/graph_kernel.pyx 中用Cython语言实现了节点采样函数node2vec_sample，请试着用numpy实现自己的node2vec_sample函数吧

In [6]:
%%writefile userdef_sample.py

import numpy as np

def node2vec_sample(succ, prev_succ, prev_node, p, q):
    """
    输入：succ - 当前节点的下一个相邻节点id列表 list (num_neighbors,)
         prev_succ - 前一个节点的下一个相邻节点id列表 list (num_neighbors,)
         prev_node - 前一个节点id int
         p - 控制回到上一节点的概率 float
         q - 控制偏向DFS还是BFS float
    输出：下一个节点id int
    """
    ##################################
    # 请在此实现node2vec的节点采样函数
     get_indexs = lambda xs, x:[i for (y, i) in zip(xs, range(len(xs))) if x==y]
     succ_len = len(succ)
     prev_succ_len = len(prev_succ)
     prev_succ_set = set()
     probs = []
     prob_sum = 0
     sampled_succ = 0

     for i in range(prev_succ_len):
        prev_succ_set.add(prev_succ[i])

     for i in range(succ_len):
        if succ[i] == prev_node:
            prob = 1. / p
        elif get_indexs(prev_succ_set, succ[i]) != len(prev_succ_set)-1:
            prob = 1.
        else:
            prob = 1. / q
        probs.append(prob)
        prob_sum += prob

     rand_num = random.random()*prob_sum
      for i in range(succ_len):
        rand_num -= probs[i]
        if rand_num <= 0:
            sampled_succ = succ[i]

    ################################## 

    return sampled_succ

Writing userdef_sample.py


In [7]:
!python my_node2vec.py  --use_my_sample --epoch 5 # 用自己实现的采样函数训练模型
!python link_predict.py --ckpt_path ./tmp/node2vec/paddle_model --epoch 100 # 测试

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
[INFO] 2020-11-25 13:48:22,038 [my_node2vec.py:  393]:	Namespace(batch_size=512, dataset='ArXiv', epoch=5, hidden_size=128, neg_num=20, offline_learning=False, p=0.25, processes=2, q=0.25, save_path='./tmp/node2vec', use_cuda=False, use_my_model=False, use_my_sample=True, walk_len=5, win_size=5)
2020-11-25 13:48:23,071-WARNING: paddle.fluid.layers.py_reader() may be deprecated in the near future. Please use paddle.fluid.io.DataLoader.from_generator() instead.
2020-11-25 13:48:23,101-WARNING: Your decorated reader has raised an exception!
Exception in thread Thread-1:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()